In [13]:
import nltk
import pandas as pd
import numpy as np
from nltk import sent_tokenize,word_tokenize
#Get lists of positive and negative words
def get_words(url):
    import requests
    words = requests.get(url).content.decode('latin-1')
    word_list = words.split('\n')
    index = 0
    while index < len(word_list):
        word = word_list[index]
        if ';' in word or not word:
            word_list.pop(index)
        else:
            index+=1
    return word_list

p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
positive_words = get_words(p_url)
negative_words = get_words(n_url)

In [15]:
def get_pos_neg_ratio(sent, positive_words, negative_words):
    cpos=0
    cneg=0
    for word in sent:
        if len(word) > 1:
            if not word[1] in ['JJ','JJR','JJS']:
                continue
            if word[0] in positive_words:
                cpos += 1
            if word[0] in negative_words:
                cneg += 1
    total_words = len(sent)
    ppct = cpos/total_words *100
    npct = cneg/total_words * 100
    if cneg == 0:
        p_n_t = 0
    else:
        p_n_t = cpos/cneg
    return ppct,npct,p_n_t

def get_entity_names(text):
    sentences = nltk.sent_tokenize(text)

    def extract_entity_names(t):
        entity_names = []

        if hasattr(t, 'label') and t.label:
            if t.label() == 'NE':
                entity_names.append(' '.join([child[0] for child in t]))
            else:
                for child in t:
                    entity_names.extend(extract_entity_names(child))

        return entity_names
            

    entity_sent_sentiment = pd.DataFrame(columns=['sentence','entity names', 'postive ratio', 'negative ratio', 'pos/neg tatio'])
    for sentence in sentences:
        tokenized_sentence = nltk.word_tokenize(sentence)
        tagged_sentence = nltk.pos_tag(tokenized_sentence)
        chunked_sentence = nltk.ne_chunk_sents([tagged_sentence], binary=True)
        for tree in chunked_sentence:
            print(tree)
            name_entity = extract_entity_names(tree)
            if len(name_entity) > 0:
                ppct,npct,p_n_t = get_pos_neg_ratio(tree, positive_words, negative_words)
                entity_sent_sentiment = entity_sent_sentiment.append({'sentence':sentence,'entity names':set(name_entity), 'postive ratio': ppct, 'negative ratio': npct, 'pos/neg tatio': p_n_t},ignore_index=True) 
            else:
                
                entity_sent_sentiment = entity_sent_sentiment.append({'sentence':sentence,'entity names':np.NaN, 'postive ratio':np.NaN, 'negative ratio':np.NaN, 'pos/neg tatio':np.NaN},ignore_index=True) 

    return entity_sent_sentiment

In [16]:
import os
x = os.listdir('./news')
for files in x:
    text_file = open('./news/'+files, "r").read()
    text_pd = get_entity_names(text_file)
    text_pd.to_csv('./sent/'+files[:-3]+'csv')

(S
  Prices/NNS
  Recent/JJ
  sales/NNS
  Freight/VBD
  2/CD
  3/CD
  3/CD
  Nitrogen/NNP
  Phosphates/NNP
  Ammonia/NNP
  Sulphur/NNP
  Potash/NNP
  4/CD
  9/CD
  11/CD
  16/CD
  19/CD
  Plant/NNP
  problems/NNS
  deal/VBP
  impairment/JJ
  blow/NN
  20/CD
  to/TO
  Yara/VB
  21/CD
  Pemex/NNP
  acquires/NNS
  (NE Grupo/NNP Fertinal/NNP)
  PIC/NNP
  mulls/VBZ
  shutting/VBG
  ferts/NNS
  operations/NNS
  21/CD
  YPFB/NNP
  signs/NNS
  ferts/NNS
  deal/NN
  with/IN
  Petrobras/NNP
  21/CD
  Fertilizers/NNS
  at/IN
  a/DT
  glance/NN
  Nitrogen/NNP
  high-margin/NN
  markets/NNS
  on/IN
  weak/JJ
  DAP/MAP/NNP
  demand/NN
  globally/RB
  ./.)
(S
  Feb/NN
  2013/CD
  Feb/NNP
  2014/CD
  Feb/NNP
  2015/CD
  Feb/NNP
  2016/CD
  1/CD
  Sulphur/NNP
  ■/NNP
  Tasweeq/NNP
  ,/,
  (NE ADNOC/NNP)
  announce/NN
  lower/JJR
  February/NNP
  prices/NNS
  (NE Tasweeq/NNP)
  has/VBZ
  decreased/VBN
  its/PRP$
  Qatar/NNP
  Sulphur/NNP
  Price/NNP
  (/(
  (NE QSP/NNP)
  )/)
  for/IN
  February/NNP
  b